In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [13]:
class VitInputLayer(nn.Module):
  def __init__(
      self,
      in_channels:int=3,
      emb_dim:int=384,
      num_patch_row:int=2,
      image_size:int=32
      ):

    '''
      in_channels:入力画像のチャネル数
      emb_dim:埋め込み後のベクトルの長さ
      num_patch_row:高さ方向のバッチの数
      image_size:入力画像の1辺の長さ
    '''
    super().__init__()
    self.in_channels = in_channels
    self.emb_dim = emb_dim
    self.num_patch_row = num_patch_row
    self.image_size = image_size

    # パッチの面積
    self.num_patch = self.num_patch_row * self.num_patch_row
    # パッチの大きさ(画像サイズ/パッチ1辺の長さ)
    self.patch_size = int(self.image_size / self.num_patch_row)

    # 入力画像のパッチ分割 & パッチの埋め込み
    self.patch_emb_layer = nn.Conv2d(
        in_channels=self.in_channels,
        out_channels=self.emb_dim,
        kernel_size=self.patch_size,
        stride=self.patch_size
    )

    # クラストークン
    self.cls_token = nn.Parameter(torch.rand(1,1,emb_dim))

    # 位置埋め込み
    self.pos_emb = nn.Parameter(torch.rand(1,self.num_patch+1,emb_dim))

  def forward(self,x:torch.Tensor) -> torch.Tensor:
    '''
      引数:
        x:入力画像.(B,C,H,W)

      返り値:
        z_0:ViTに入力する特徴量.(B,N,emb_dim)
          B:パッチサイズ　N:トークン数　emb_dim:埋め込み後のベクトルの長さ
    '''

    # (B,C,H,W) -> (B,D,H/P,W/P)
    # P:バッチ1辺の長さ
    z_0 = self.patch_emb_layer(x)
    print(z_0.shape)
    # パッチのflatten. (B,D,H/P,W/P) -> (B,D,Np)
    # Npはパッチの数(H*W/P^2)
    # 2を指定することで2次元目から後ろの次元をすべて1次元にまとめる
    z_0 = z_0.flatten(2)

    # 軸の入れ替え (B,D,Np) -> (B,Np,D)
    z_0 = z_0.transpose(1,2)

    # 埋め込みの先頭にクラストークンを結合
    # (B,Np,D) -> (B,Np+1,D)
    # クラストークンは(1,1,D)なのでリピートで(B,1,D)
    z_0 = torch.cat([self.cls_token.repeat(repeats=(x.shape[0],1,1)),z_0],dim=1)

    # 位置埋め込み
    z_0 = z_0 + self.pos_emb

    return z_0

In [14]:
batch_size,channel,height,width=2,3,32,32
x=torch.rand(batch_size,channel,height,width)
vit_input_layer=VitInputLayer()
z_0=vit_input_layer(x)
print(z_0.shape)

torch.Size([2, 384, 2, 2])
torch.Size([2, 5, 384])


In [15]:
class MultiHeadSelfAttention(nn.Module):
  def __init__(self,emb_dim:int=384,head:int=3,dropout:float=0.0):
    '''
      emb_dim:埋め込み後のベクトルの長さ
      head:ヘッドの数
      dropout:ドロップアウト率
    '''
    super().__init__()
    self.head = head
    self.emb_dim = emb_dim
    self.head_dim = emb_dim//head
    self.sqrt_dh = self.head_dim**0.5 # Dhの二乗根

    # 入力をq,k,vに埋め込むための線形層
    self.w_q = nn.Linear(emb_dim,emb_dim,bias=False)
    self.w_k = nn.Linear(emb_dim,emb_dim,bias=False)
    self.w_v = nn.Linear(emb_dim,emb_dim,bias=False)
    self.attn_drop = nn.Dropout(dropout)


    # MHSAを埋め込むための線形層
    self.w_o = nn.Sequential(
        nn.Linear(emb_dim,emb_dim),
        nn.Dropout(dropout)
    )

  def forward(self,z:torch.Tensor) -> torch.Tensor:
    '''
      引数:
        z:MHSAに入力する特徴量.(B,N,D)
          B:バッチサイズ　N:トークン数　D:ベクトルの長さ
      返り値:
        z:MHSAを出力する特徴量.(B,N,D)
          B:パッチサイズ　N:トークン数　emb_dim:埋め込み後のベクトルの長さ
    '''

    batch_size,num_patch,_ = z.shape

    # 埋め込み
    q = self.w_q(z)
    k = self.w_k(z)
    v = self.w_v(z)

    # ヘッドに分ける
    # (B,N,D) -> (B,N,h,D//h)
    q = q.view(batch_size,num_patch,self.head,self.head_dim)
    k = k.view(batch_size,num_patch,self.head,self.head_dim)
    v = v.view(batch_size,num_patch,self.head,self.head_dim)

    # 形の変更(ヘッドごとに操作するため)
    # (B,N,h,D//h) -> (B,h,N,D//h)
    q = q.transpose(1,2)
    k = k.transpose(1,2)
    v = v.transpose(1,2)

    # ヘッドの転置(行列積を取るため)
    # (B,h,N,D//h) -> (B,h,D//h,N)
    k_T=k.transpose(2,3)

    # (B, h, N, D//h) x (B, h, D//h, N) -> (B, h, N, N)
    dots = (q @ k_T) / self.sqrt_dh

    # 列方向にsoftmax
    attn = F.softmax(dots,dim=-1)

    # ドロップアウト
    attn = self.attn_drop(attn)


    # 加重和
    # (B, h, N, N) x (B, h, N, D//h) -> (B, h, N, D//h)
    out = attn @ v

    # (B, h, N, D//h) -> (B, N, h, D//h) (headを結合するため)
    out = out.transpose(1,2)

    # (B, N, h, D//h) -> (B,N,D)　結合
    out = out.reshape(batch_size,num_patch,self.emb_dim)

    # 出力層
    out = self.w_o(out)

    return out

In [16]:
mhsa = MultiHeadSelfAttention()
out = mhsa(z_0)

print(z_0.shape)

torch.Size([2, 5, 384])


In [18]:
class ViTEncoderBlock(nn.Module):
  def __init__(self,emb_dim:int=384,head:int=3,hidden_dim:int=384*4,dropout:float=0.0):
    '''
      emb_dim:埋め込み後のベクトルの長さ
      head:ヘッドの数
      hidden_dim:隠れ層の次元 (MLPにおける中間層のベクトルの長さ)
      dropout:ドロップアウト率
    '''
    super().__init__()
    self.ln1 = nn.LayerNorm(emb_dim)
    self.msa = MultiHeadSelfAttention(emb_dim=emb_dim,head=head,dropout=dropout)

    self.ln2 = nn.LayerNorm(emb_dim)
    self.mlp = nn.Sequential(
        nn.Linear(emb_dim,hidden_dim),
        nn.GELU(),
        nn.Dropout(dropout),
        nn.Linear(hidden_dim,emb_dim),
        nn.Dropout(dropout)
    )

  def forward(self,z: torch.Tensor) -> torch.Tensor:
    '''
      引数:
        z:ViTEncoderBlockに入力する特徴量.(B,N,D)
          B:バッチサイズ　N:トークン数　D:ベクトルの長さ
      返り値:
        out:ViTEncoderBlockへの出力する特徴量.(B,N,D)
          B:バッチサイズ　N:トークン数　D:ベクトルの長さ
  　'''
    out = self.msa(self.ln1(z)) + z
    out = self.mlp(self.ln2(out)) + out

    return out

In [19]:
vit_enc = ViTEncoderBlock()
out = vit_enc(z_0)
print(out.shape)

torch.Size([2, 5, 384])


In [20]:
class ViT(nn.Module):
  def __init__(self,in_channels:int=3,num_classes:int=10,emb_dim:int=384,num_patch_row:int=2,image_size:int=32,num_blocks:int=7,head:int=8,hidden_dim:int=384*4,dropout:float=0.0):
    '''
      in_channels:入力画像のチャネル数
      num_classes:分類クラスの数
      emb_dim:埋め込み後のベクトルの長さ
      num_patch_row:高さ方向のバッチの数
      image_size:入力画像の1辺の長さ
      num_blocks:ViTのブロック数
      head:ヘッドの数
      hidden_dim:隠れ層の次元 (MLPにおける中間層のベクトルの長さ)
      dropout:ドロップアウト率
    '''

    super().__init__()
    self.input_layer = VitInputLayer(in_channels=in_channels,emb_dim=emb_dim,num_patch_row=num_patch_row,image_size=image_size)

    self.encoder = nn.Sequential(*[ViTEncoderBlock(emb_dim=emb_dim,head=head,hidden_dim=hidden_dim,dropout=dropout) for _ in range(num_blocks)])

    self.mlp_head = nn.Sequential(
        nn.LayerNorm(emb_dim),
        nn.Linear(emb_dim,num_classes)
    )

  def forward(self,x:torch.Tensor) -> torch.Tensor:
    '''
      引数:
        x:ViTに入力する特徴量.(B,C,H,W)
          B:バッチ C:チャネル数 H:高さ W:幅
      返り値:
        out:ViTの出力する特徴量.(B,num_classes)
          B:バッチ num_classes:分類クラスの数
    '''
    out = self.input_layer(x)
    out = self.encoder(out)
    cls_token = out[:,0] # 最初の行だけ取り出す
    out = self.mlp_head(cls_token)

    return out


In [21]:
num_classes = 10
batch_size,channel,height,width=2,3,32,32
x=torch.rand(batch_size,channel,height,width)
vit = ViT(in_channels=channel,num_classes=num_classes)
out = vit(x)
print(out.shape)

torch.Size([2, 384, 2, 2])
torch.Size([2, 10])
